In [2]:
import numpy as np
import pandas as pd

In [3]:
train_data_eq_prior = []
test_data_eq_prior = []
train_data_noneq_prior = []
test_data_noneq_prior = []

est_mean_eq_prior = [[0,0],[0,0]]
est_mean_noneq_prior = [[0,0],[0,0]]
est_var_eq_prior = [[[0,0],[0,0]],[[0,0],[0,0]]]
est_var_noneq_prior = [[[0,0],[0,0]],[[0,0],[0,0]]]
est_prior_eq_prior = [0,0]
est_prior_noneq_prior = [0,0]
train_count_eq_prior = [0,0]
test_count_eq_prior = [0,0]
train_count_noneq_prior = [0,0]
test_count_noneq_prior = [0,0]


mean = [[0.1,0.8],[0.5,0.2]]
covar = [[[1,0],[0,1]],[[1, 1.5],[0, -1 ]]]

data1 = np.random.choice(2, 50000, p=[0.5,0.5])
data2 = np.random.choice(2, 10000, p=[0.5,0.5])
data3 = np.random.choice(2, 50000, p=[0.2,0.8])
data4 = np.random.choice(2, 10000, p=[0.2,0.8])

for i in range(50000):
    train_data_eq_prior.append([np.random.multivariate_normal(mean[data1[i]],covar[data1[i]]),data1[i]])
    train_data_noneq_prior.append([np.random.multivariate_normal(mean[data3[i]],covar[data3[i]]),data3[i]])
    train_count_eq_prior[data1[i]]+= 1
    train_count_noneq_prior[data3[i]] += 1

for i in range(10000):
    test_data_eq_prior.append([np.random.multivariate_normal(mean[data2[i]],covar[data2[i]]),data2[i]])
    test_data_noneq_prior.append([np.random.multivariate_normal(mean[data4[i]],covar[data4[i]]),data4[i]])
    test_count_eq_prior[data2[i]]+= 1
    test_count_noneq_prior[data4[i]] += 1


for i in range(50000):
    est_mean_eq_prior[train_data_eq_prior[i][1]] += train_data_eq_prior[i][0]
    est_mean_noneq_prior[train_data_noneq_prior[i][1]] += train_data_noneq_prior[i][0]
    est_prior_eq_prior[train_data_eq_prior[i][1]] += 1
    est_prior_noneq_prior[train_data_noneq_prior[i][1]] += 1

for i in range(2):
    est_mean_eq_prior[i] /= est_prior_eq_prior[i]
    est_mean_noneq_prior[i] /= est_prior_noneq_prior[i]

for i in range(50000):
    mat1 = np.matrix((train_data_eq_prior[i][0]-est_mean_eq_prior[train_data_eq_prior[i][1]]))
    est_var_eq_prior[train_data_eq_prior[i][1]] += np.matmul(mat1.transpose(),mat1)
    mat2 = np.matrix((train_data_noneq_prior[i][0]-est_mean_noneq_prior[train_data_noneq_prior[i][1]]))
    est_var_noneq_prior[train_data_noneq_prior[i][1]] += np.matmul(mat2.transpose(),mat2)

for i in range(2):
    est_var_eq_prior[i] /= est_prior_eq_prior[i]
    est_var_noneq_prior[i] /= est_prior_noneq_prior[i]
    est_prior_eq_prior[i] /= 50000
    est_prior_noneq_prior[i] /= 50000

/home/ankit/yes/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: covariance is not positive-semidefinite.
/home/ankit/yes/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: covariance is not positive-semidefinite.
/home/ankit/yes/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: covariance is not positive-semidefinite.
/home/ankit/yes/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: covariance is not positive-semidefinite.


In [4]:
# equal priors and variances
def makePred_eq_priors_eq_var(trainData,testData,est_mean):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            mat1 = np.matrix(testData[i][0])
            res = [est_mean[0]*mat1.transpose() - np.linalg.norm(est_mean[0])/2, est_mean[1]*mat1.transpose() - np.linalg.norm(est_mean[1])/2]
            res = res.index(max(list(res)))
            pred_test[i] = res
        mat1 = np.matrix(trainData[i][0])
        res = [est_mean[0]*mat1.transpose() - np.linalg.norm(est_mean[0])/2, est_mean[1]*mat1.transpose() - np.linalg.norm(est_mean[1])/2]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test

In [12]:
# estimated priors and variances
# can improve using the associativity of matrix multiplication
def makePred_est_priors_est_var(trainData,testData,est_mean,est_covar,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        m_0 = np.matrix(est_mean[0])
        m_1 = np.matrix(est_mean[1])
        S_0 = np.matrix(est_covar[0])
        S_1 = np.matrix(est_covar[1])
        if i < 10000:
            mat1 = np.matrix(testData[i][0])
            r1 = np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),mat1.transpose())
            r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),m_0.transpose())
            r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S_0)),m_0.transpose())
            r4 = np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),mat1.transpose())
            r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),m_1.transpose())
            r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S_1)),m_1.transpose())
            res = [-np.log(np.linalg.det(S_0))/2 - (r1-r2+r3)/2 + np.log(est_prior[0]),
                   -np.log(np.linalg.det(S_1))/2 - (r4-r5+r6)/2 + np.log(est_prior[1])]
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        mat1 = np.matrix(trainData[i][0])
        r1 = np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),mat1.transpose())
        r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_0)),m_0.transpose())
        r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S_0)),m_0.transpose())
        r4 = np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),mat1.transpose())
        r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S_1)),m_1.transpose())
        r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S_1)),m_1.transpose())
        res = [-np.log(np.linalg.det(S_0))/2 - (r1-r2+r3)/2 + np.log(est_prior[0]),
               -np.log(np.linalg.det(S_1))/2 - (r4-r5+r6)/2 + np.log(est_prior[1])]
        
        #print("RES: ",list(res))
        res = res.index(max(list(res)))
        
        pred_train[i] = res
    return pred_train, pred_test 

In [6]:
common_covar_eq_prior = [[0,0],[0,0]]
common_covar_noneq_prior = [[0,0],[0,0]]
for i in range(len(covar)):
    common_covar_eq_prior += est_prior_eq_prior[i]*est_var_eq_prior[i]
    common_covar_noneq_prior += est_prior_noneq_prior[i]*est_var_noneq_prior[i]

print(common_covar_eq_prior)
print(common_covar_noneq_prior)

[[0.89624293 0.30425097]
 [0.30425097 1.35331479]]
[[0.84107272 0.48314655]
 [0.48314655 1.56258203]]


In [5]:
# using common covariance and estimated priors
def makePred_common_covar_est_prior(trainData,testData,est_mean,com_covar,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        m_0 = np.matrix(est_mean[0])
        m_1 = np.matrix(est_mean[1])
        S = np.matrix(com_covar)
        if i < 10000:
            mat1 = np.matrix(testData[i][0])
            r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_0.transpose())
            r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S)),m_0.transpose())
            r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_1.transpose())
            r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S)),m_1.transpose())
            res = [- (-r2+r3)/2 + np.log(est_prior[0]), - (-r5+r6)/2 + np.log(est_prior[1])]
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        mat1 = np.matrix(trainData[i][0])
        r2 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_0.transpose())
        r3 = np.matmul(np.matmul(m_0,np.linalg.inv(S)),m_0.transpose())
        r5 = 2*np.matmul(np.matmul(mat1,np.linalg.inv(S)),m_1.transpose())
        r6 = np.matmul(np.matmul(m_1,np.linalg.inv(S)),m_1.transpose())
        res = [-(-r2+r3)/2 + np.log(est_prior[0]), -(-r5+r6)/2 + np.log(est_prior[1])]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test  

In [10]:
def calcAccuracy(trainLabel_DIST1,count_train,pred_train,testLabel_DIST1,count_test,pred_test):
    train_accuracy = [0,0]
    test_accuracy = [0,0]
    for i in range(50000):
        if i < 10000:
            if testLabel_DIST1[i][1] == pred_test[i]:
                test_accuracy[testLabel_DIST1[i][1]] += 1
        if trainLabel_DIST1[i][1] == pred_train[i]:
            train_accuracy[trainLabel_DIST1[i][1]] += 1

    for i in range(2):
        train_accuracy[i] /= count_train[i]
        test_accuracy[i] /= count_test[i]
    return train_accuracy, test_accuracy

In [6]:
# Naive-Bayes Classifier
def makePred_naive_bayes(trainData,testData,est_mean,est_covar,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000

    for i in range(50000):
        if i < 10000:
            res = [0,0]
            for j in range(2):
                tmp = 0
                for k in range(2):
                    tmp += np.power((testData[i][0][k]-est_mean[j][k]),2)/est_covar[k,k]
                res[j] += -tmp/2 + np.log(est_prior[j])
            res = res.index(max(list(res)))
            pred_test[i] = res
            
        res = [0,0]
        for j in range(2):
            tmp = 0
            for k in range(2):
                tmp += np.power((trainData[i][0][k]-est_mean[j][k]),2)/est_covar[k,k]
            res[j] += -tmp/2 + np.log(est_prior[j])
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test  

In [10]:
# equal variances est. priors
def makePred_est_priors_eq_Allvar(trainData,testData,est_mean,var,est_prior):
    pred_train = [None]*50000
    pred_test = [None]*10000
    mean0 = np.matrix(est_mean[0])
    mean1 = np.matrix(est_mean[1])

    for i in range(50000):
        if i < 10000:
            mat1 = np.matrix(testData[i][0])
            res = [-np.linalg.norm(mat1-mean0)/(2*var)+np.log(est_prior[0]), -np.linalg.norm(mat1-mean1)/(2*var) + np.log(est_prior[1])]
            res = res.index(max(list(res)))
            pred_test[i] = res
        mat1 = np.matrix(trainData[i][0])
        res = [-np.linalg.norm(mat1-mean0)/(2*var)+np.log(est_prior[0]), -np.linalg.norm(mat1-mean1)/(2*var) + np.log(est_prior[1])]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test

In [11]:
# equal variances equal priors
def makePred_eq_priors_eq_Allvar(trainData,testData,est_mean,var):
    pred_train = [None]*50000
    pred_test = [None]*10000
    mean0 = np.matrix(est_mean[0])
    mean1 = np.matrix(est_mean[1])

    for i in range(50000):
        if i < 10000:
            mat1 = np.matrix(testData[i][0])
            res = [-np.linalg.norm(mat1-mean0), -np.linalg.norm(mat1-mean1)]
            res = res.index(max(list(res)))
            pred_test[i] = res
        mat1 = np.matrix(trainData[i][0])
        res = [-np.linalg.norm(mat1-mean0), -np.linalg.norm(mat1-mean1)]
        res = res.index(max(list(res)))
        pred_train[i] = res
    return pred_train, pred_test

In [23]:
pred_train1, pred_test1 = makePred_eq_priors_eq_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior)
t1 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)
t1

([0.6684959834566134, 0.6313269493844049],
 [0.6566558441558441, 0.6301261829652997])

In [13]:
pred_train1, pred_test1 = makePred_est_priors_est_var(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,est_var_eq_prior,est_prior_eq_prior)
t2 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)

In [25]:
pred_train1, pred_test1 = makePred_common_covar_est_prior(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,common_covar_eq_prior,est_prior_eq_prior)
t3 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)

In [26]:
pred_train1, pred_test1 = makePred_naive_bayes(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,common_covar_eq_prior,est_prior_eq_prior)
t4 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)

In [27]:
s = np.power(np.linalg.det(np.matrix(common_covar_eq_prior)),1/2)
pred_train1, pred_test1 = makePred_est_priors_eq_Allvar(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,s,est_prior_eq_prior)
t5 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)

In [28]:
pred_train1, pred_test1 = makePred_eq_priors_eq_Allvar(train_data_eq_prior,test_data_eq_prior,est_mean_eq_prior,s)
t6 = calcAccuracy(train_data_eq_prior,train_count_eq_prior,pred_train1,test_data_eq_prior,test_count_eq_prior,pred_test1)

In [39]:
results = pd.DataFrame({'eq_priors_eq_var':t1,
                        'est_priors_est_var':t2,
                        'common_covar_est_prior':t3,
                        'naive_bayes':t4,
                        'est_priors_eq_Allvar':t5,
                        'eq_priors_eq_Allvar':t6})
print("0th row shows train error and 1th row shows test error.\n",
      "For each row, the errors are shown class-wise.\n")
print("RESULTS FOR DATA WITH EQUAL PRIORS")
results

0th row shows train error and 1th row shows test error.
 For each row, the errors are shown class-wise.

RESULTS FOR DATA WITH EQUAL PRIORS


,eq_priors_eq_var,est_priors_est_var,common_covar_est_prior,naive_bayes,est_priors_eq_Allvar,eq_priors_eq_Allvar
0,"[0.6649557165861514, 0.6279452875399361]","[0.23389694041867956, 0.9767871405750799]","[0.17159822866344607, 0.9859474840255591]","[0.02848228663446055, 0.9996505591054313]","[0.0, 1.0]","[0.6413043478260869, 0.6540784744408946]"
1,"[0.6646433990895296, 0.6253271843450081]","[0.2402630247850278, 0.9766920104698991]","[0.17754172989377845, 0.9864140595787112]","[0.030854830551340414, 0.9996260750342765]","[0.0, 1.0]","[0.6363176530096105, 0.6527483484980681]"


In [37]:
pred_train1, pred_test1 = makePred_eq_priors_eq_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior)
t1 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t1

([0.6649557165861514, 0.6279452875399361],
 [0.6646433990895296, 0.6253271843450081])

In [31]:
pred_train1, pred_test1 = makePred_est_priors_est_var(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,est_var_noneq_prior,est_prior_noneq_prior)
t2 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t2

([0.23389694041867956, 0.9767871405750799],
 [0.2402630247850278, 0.9766920104698991])

In [32]:
pred_train1, pred_test1 = makePred_common_covar_est_prior(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,common_covar_noneq_prior,est_prior_noneq_prior)
t3 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t3

([0.17159822866344607, 0.9859474840255591],
 [0.17754172989377845, 0.9864140595787112])

In [33]:
pred_train1, pred_test1 = makePred_naive_bayes(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,common_covar_noneq_prior,est_prior_noneq_prior)
t4 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t4

([0.02848228663446055, 0.9996505591054313],
 [0.030854830551340414, 0.9996260750342765])

In [34]:
s = np.power(np.linalg.det(np.matrix(common_covar_noneq_prior)),1/2)
pred_train1, pred_test1 = makePred_est_priors_eq_Allvar(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,s,est_prior_noneq_prior)
t5 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t5

([0.0, 1.0], [0.0, 1.0])

In [35]:
pred_train1, pred_test1 = makePred_eq_priors_eq_Allvar(train_data_noneq_prior,test_data_noneq_prior,est_mean_noneq_prior,s)
t6 = calcAccuracy(train_data_noneq_prior,train_count_noneq_prior,pred_train1,test_data_noneq_prior,test_count_noneq_prior,pred_test1)
t6

([0.6413043478260869, 0.6540784744408946],
 [0.6363176530096105, 0.6527483484980681])

In [40]:
results1 = pd.DataFrame({'eq_priors_eq_var':t1,
                        'est_priors_est_var':t2,
                        'common_covar_est_prior':t3,
                        'naive_bayes':t4,
                        'est_priors_eq_Allvar':t5,
                        'eq_priors_eq_Allvar':t6})
print("0th row shows train error and 1th row shows test error.\n",
      "For each row, the errors are shown class-wise.\n")
print("RESULTS FOR DATA WITH NON-EQUAL PRIORS")
results1

0th row shows train error and 1th row shows test error.
 For each row, the errors are shown class-wise.

RESULTS FOR DATA WITH NON-EQUAL PRIORS


,eq_priors_eq_var,est_priors_est_var,common_covar_est_prior,naive_bayes,est_priors_eq_Allvar,eq_priors_eq_Allvar
0,"[0.6649557165861514, 0.6279452875399361]","[0.23389694041867956, 0.9767871405750799]","[0.17159822866344607, 0.9859474840255591]","[0.02848228663446055, 0.9996505591054313]","[0.0, 1.0]","[0.6413043478260869, 0.6540784744408946]"
1,"[0.6646433990895296, 0.6253271843450081]","[0.2402630247850278, 0.9766920104698991]","[0.17754172989377845, 0.9864140595787112]","[0.030854830551340414, 0.9996260750342765]","[0.0, 1.0]","[0.6363176530096105, 0.6527483484980681]"


### Note: It is just for a change that we looked at the accuracies for individual classes instead of overall accuracies.